## Create merged file for dashboard
Merge sentiment_client_agent.csv with the LDA classification output.

In [21]:
import codecs, os
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats

data_folder = '/home/daniel/s2ds/Data/'
client_agent_file = 'sentiment_client_agent.csv'
ms_topics_file = '05_fastTextConv_MStopicId.txt'
conv_file = 'dashboard.json'

Build dict with convID being the key and LDA predicted topic the value

In [22]:
MS_topic_labels = \
{ 0:'Other reason'\
, 1:'Dropped chat'\
, 2:'Promotion code'\
, 3:'Sparks card'\
, 4:'Live chat'\
, 5:'Placing order'\
, 6:'Amending order'\
, 7:'Delivery'\
, 8:'Food'\
, 9:'Password reset'\
,10:'Policy queries'\
,11:'GM availability'\
,12:'Store feedback'\
,13:'Dropped call'\
,14:'GM quality'\
,15:'Returns/refunds'\
,16:'Transfer'\
,17:'Website feedback'\
,18:'Password  reset'}

ms_topics_file = codecs.open(os.path.join(data_folder, ms_topics_file), 'rU', 'utf-8')
ms_topics = {}

for i, line in enumerate(ms_topics_file):
    i
    ms_topics[i] = MS_topic_labels[int(line.strip())]

Add LDA topics as new column to filtered summary

In [24]:
summary = pd.read_csv(os.path.join(data_folder, client_agent_file), index_col=0)
sentiment = summary[['client', 'agent', 'convIDFiltered']]
# get rid of rows/conversations which were filtered out by the preprocessing 
# and also multiple rows within the same conv (corresponding diff messages)
filter_ind = ~pd.isnull(summary.convIDFiltered.drop_duplicates())
# tricky indexing, didn't find a better way to do this efficiently
summary = summary.loc[filter_ind.index[np.where(filter_ind)[0]]]
# turn lda numbers into lda topic labels
lda_topics = [ms_topics[convID] for convID in summary.convIDFiltered.values]
summary.insert(summary.shape[1], 'msTopic', lda_topics)
# discard stuff we don't need
summary = summary[['convIDFiltered', 'convLen', 'convDate', 'convSec', 'agentID', 'msTopic']]
# add new human readable column names
summary.columns = ['MS_ID', 'Messages', 'Date', 'Seconds', 'AgentID', 'Topic']
# dirty trick to get rid of 3 missing values in AgentID column
summary.iloc[np.where(pd.isnull(summary.AgentID))[0], 4] = ['c1817483'] * 3

Add columns for date (month, day, hour)

In [25]:
# this dict holds the min, median and max of all continuous cols in summary
min_med_max = {}

# get min, med, max dates
date = pd.to_datetime(summary.Date.values.ravel(), errors='coerce',infer_datetime_format=True)
min_date = date.min()
delta_date = pd.to_timedelta((date - min_date).astype('m8[ms]').to_series().median(), unit='ms')
median_date = min_date + delta_date
max_date = date.max()
min_med_max['Date'] = {
    'min': min_date,
    'median': median_date,
    'max': max_date
}

# replace original date var
summary.drop('Date', 1, inplace=True)
summary.insert(summary.shape[1], 'Date', pd.Series(date).values)

# add month
date = pd.to_datetime(summary.Date)
months = {
    '1': 'Jan',
    '2': 'Feb',
    '3': 'Mar',
    '4': 'Apr',
    '5': 'May',
    '6': 'Jun',
    '7': 'Jul',
    '8': 'Aug',
    '9': 'Sep',
    '10': 'Oct',
    '11': 'Nov',
    '12': 'Dec',
}
months = [months[str(m)] for m in date.dt.month.values]
summary.insert(summary.shape[1], 'Month', months)

# add weekday
weekdays = {
    '0': 'Mon',
    '1': 'Tue',
    '2': 'Wed',
    '3': 'Thu',
    '4': 'Fri',
    '5': 'Sat',
    '6': 'Sun'
}
weekdays = [weekdays[str(m)] for m in date.dt.dayofweek.values]
summary.insert(summary.shape[1], 'Day', weekdays)

# add hour of conversation
summary.insert(summary.shape[1], 'Hour', date.dt.hour.values)

Add columns for sentiment

In [27]:
from joblib import Parallel, delayed

# find indices of first and last messages in each conversation
ind_first = sentiment.convIDFiltered.drop_duplicates(keep='first')
ind_first = ind_first[~pd.isnull(ind_first)].index
ind_last = sentiment.convIDFiltered.drop_duplicates(keep='last')
ind_last = ind_last[~pd.isnull(ind_last)].index

# get sentiment of first and last sentence
first_sentiment = sentiment.client.loc[ind_first].values
last_sentiment = sentiment.client.loc[ind_last].values

# fit a linear line in parallel to sentiment to get an overall trend of conv sentiment
def get_sentiment_fit(i):
    data = sentiment[sentiment.convIDFiltered == i]
    y = data.client
    x = range(data.shape[0])
    r = stats.spearmanr(x, y)[0]
    if np.isnan(r):
        r = 0
    return r

num_conv = sentiment.convIDFiltered.max() + 1
sentiment_fit = np.array(Parallel(n_jobs=-1)(delayed(get_sentiment_fit)(i) for i in np.arange(num_conv)))

In [28]:
summary.insert(summary.shape[1], 'SentimentFirst', first_sentiment)
summary.insert(summary.shape[1], 'SentimentLast', last_sentiment)
summary.insert(summary.shape[1], 'SentimentFit', sentiment_fit)

Filter the dataset discard very long and very rare chats

In [29]:
summary = summary[summary.Messages < 21]
summary.Seconds = summary.Seconds

Save file as json and make it human readable

In [30]:
# change MS_ID to int before save
# summary.MS_ID = range(1, summary.shape[0] + 1)
# save it as json
json_file = '/home/daniel/ms/Daniel/dashboard/ms.json'
summary.to_json(os.path.join(json_file), orient='records', date_format='iso')
# make sure the json is human readable, pandas output is just one very long line
f = open(json_file, 'r')
overwrite = f.read().replace('},{' , '},\n{')
f = open(json_file, 'w')
f.write(overwrite)
f.close()

Add min median and max values for continous variables and print them so we can insert it into JS

In [31]:
cont_cols = ['Messages', 'Seconds', 'Hour', 'SentimentFirst', 'SentimentLast', 'SentimentFit']
for col in cont_cols:
    min_med_max[col] = {
        'min': summary[col].min() - abs(summary[col].min()) * .05,
        'median': summary[col].median(),
        'max': summary[col].max() + abs(summary[col].max()) * .05
    }

In [32]:
for k, v in min_med_max.iteritems():
    print k
    print v['min']
    print v['median']
    print v['max']
    print '--------------------'

SentimentFit
-1.05
0.316227766017
1.05
--------------------
Hour
0.0
13.0
24.15
--------------------
SentimentLast
-1.030365
0.3818
1.049685
--------------------
Seconds
4.75
615.0
5610.15
--------------------
Messages
0.95
4.0
21.0
--------------------
Date
2016-04-01 06:00:56
2016-05-02 15:27:09
2016-06-02 14:09:41
--------------------
SentimentFirst
-1.04727
0.0
1.049685
--------------------
